[![visitor][visitor-badge]][visitor-stats]
[![ko-fi][ko-fi-badge]][ko-fi-link]

# **Kohya LoRA Trainer XL**
A Colab Notebook For SDXL LoRA Training (Fine-tuning Method)

[visitor-badge]: https://api.visitorbadge.io/api/visitors?path=Kohya%20LoRA%20Trainer%20XL&label=Visitors&labelColor=%2334495E&countColor=%231ABC9C&style=flat&labelStyle=none
[visitor-stats]: https://visitorbadge.io/status?path=Kohya%20LoRA%20Trainer%20XL
[ko-fi-badge]: https://img.shields.io/badge/Support%20me%20on%20Ko--fi-F16061?logo=ko-fi&logoColor=white&style=flat
[ko-fi-link]: https://ko-fi.com/linaqruf

| Notebook Name | Description | Link |
| --- | --- | --- |
| [Kohya LoRA Trainer XL](https://github.com/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-trainer-XL.ipynb) | LoRA Training | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=flat)](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-trainer-XL.ipynb) |
| [Kohya Trainer XL](https://github.com/Linaqruf/kohya-trainer/blob/main/kohya-trainer-XL.ipynb) | Native Training | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=flat)](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-trainer-XL.ipynb) |


<hr>
<h4><font color="#4a90e2"><b>NEWS:</b></font> <i>Colab's free-tier users can now train SDXL LoRA using the diffusers format instead of checkpoint as a pretrained model.</i></h4>
<hr>

# **I. Prepare Environment**

In [ ]:
!pip install transformers
!pip uninstall xformers -y
!pip uninstall torch torchvision -y
!pip uninstall nvidia-cublas-cu12 nvidia-cuda-cupti-cu12 nvidia-cuda-runtime-cu12 nvidia-cudnn-cu12 nvidia-cufft-cu12 nvidia-cusolver-cu12 nvidia-cusparse-cu12 -y
!pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 torchtext==0.17.0 xformers nvidia-cublas-cu12 nvidia-cuda-cupti-cu12 nvidia-cuda-runtime-cu12 nvidia-cudnn-cu12 nvidia-cufft-cu12 nvidia-cusolver-cu12 nvidia-cusparse-cu12 --no-cache-dir
!pip uninstall jax -y
!pip install jax[cuda12_pip]==0.4.23 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


In [12]:
import os
import zipfile
import shutil
import time
import requests
import torch
from subprocess import getoutput
from IPython.utils import capture

import re
import json
import glob
from urllib.parse import urlparse, unquote
from pathlib import Path
from huggingface_hub import HfFileSystem
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError
from tqdm import tqdm
from PIL import Image
import concurrent.futures
import random

In [ ]:
!pip install nvidia-cublas-cu12==12.1.3.1
!pip install nvidia-cuda-cupti-cu12==12.1.105
!pip install nvidia-cuda-runtime-cu12==12.1.105
!pip install nvidia-cudnn-cu12==8.9.2.26
!pip install nvidia-cufft-cu12==11.0.2.54
!pip install nvidia-cusolver-cu12==11.4.5.107
!pip install nvidia-cusparse-cu12==12.1.0.106


In [2]:
import os
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

# Create a DataFrame
data = {'Name': ['John', 'Emily', 'Charlie', 'Sarah', 'Michael'],
        'Age': [25, 28, 32, 29, 35]}
df = pd.DataFrame(data)

# Loop through the DataFrame and print each row with a progress bar
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing DataFrame"):
    if index == 2:
        break
    print(f"Name: {row['Name']}, Age: {row['Age']}")

# Environment directory paths setup for Kubeflow
root_dir = "/home/jovyan"
repo_dir = os.path.join(root_dir, "kohya-trainer")

repo_dict = {
    "qaneel/kohya-trainer": "https://github.com/qaneel/kohya-trainer",
    "kohya-ss/sd-scripts": "https://github.com/kohya-ss/sd-scripts",
}

repository = "qaneel/kohya-trainer"
repo_url = repo_dict[repository]
branch = "main"

def clone_repo(url, dir, branch):
    if not os.path.exists(dir):
        os.system(f"git clone -b {branch} {url} {dir}")

def setup_directories():
    dirs = [repo_dir]
    for dir in tqdm(dirs, desc="Setting up directories"):
        os.makedirs(dir, exist_ok=True)

def install_dependencies():
    os.system("pip install tqdm")
    requirements = ["requests", "numpy", "pandas"]  # Example requirements
    for req in tqdm(requirements, desc="Installing dependencies"):
        os.system(f"pip install {req}")

def prepare_environment():
    # This function prepares the environment
    print("Environment is being prepared...")

def main():
    tqdm.write("Starting the main process...")
    clone_repo(repo_url, repo_dir, branch)
    setup_directories()
    install_dependencies()
    prepare_environment()
    tqdm.write("Main process completed.")

main()


Processing DataFrame:  40%|████      | 2/5 [00:00<00:00, 3606.45it/s]


Name: John, Age: 25
Name: Emily, Age: 28
Starting the main process...


Installing dependencies: 100%|██████████| 3/3 [00:07<00:00,  2.43s/it]

Environment is being prepared...
Main process completed.


In [34]:
from ipywidgets import Button, Dropdown, Password, Output
from IPython.display import display
import requests

# Model and VAE URLs for demonstration (specifically as you provided)
MODEL_URLS = {
    "gsdf/CounterfeitXL": "https://huggingface.co/gsdf/CounterfeitXL/resolve/main/CounterfeitXL_%CE%B2.safetensors",
    "Linaqruf/animagine-xl": "https://huggingface.co/Linaqruf/animagine-xl/resolve/main/animagine-xl.safetensors",
    "stabilityai/stable-diffusion-xl-base-1.0": "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors",
    "stablediffusionapi/newreality-xl": "https://huggingface.co/stablediffusionapi/newreality-xl/blob/main/unet/diffusion_pytorch_model.bin",
    "stablediffusionapi/newrealityxl-global-nsfw": "https://huggingface.co/stablediffusionapi/newrealityxl-global-nsfw/blob/main/unet/diffusion_pytorch_model.safetensors"
}

VAE_URLS = {
    "None": "",
    "Original VAE": "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors",
    "FP16 VAE": "https://huggingface.co/madebyollin/sdxl-vae-fp16-fix/resolve/main/sdxl_vae.safetensors",
    "new reality": "https://huggingface.co/stablediffusionapi/newrealityxl-global-nsfw/blob/main/vae/diffusion_pytorch_model.safetensors"
}

# Setup widgets
model_dropdown = Dropdown(options=MODEL_URLS, description='Model:')
vae_dropdown = Dropdown(options=VAE_URLS, description='VAE:')
token_input = Password(description="Token:")
download_button = Button(description='Download')
output_area = Output()

# Display Widgets
display(model_dropdown, vae_dropdown, token_input, download_button, output_area)

def download_file(url):
    """Function to download files with streaming."""
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        # Assuming '/home/jovyan' is the path where files should be saved
        path = f"/home/jovyan/{url.split('/')[-1]}"
        with open(path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return f"Downloaded from {url}"
    else:
        return f"Failed to download from {url}"

@download_button.on_click
def on_button_clicked(b):
    """Handle button click event."""
    with output_area:
        output_area.clear_output()
        if not token_input.value:
            print("Token is required!")
        else:
            model_url = model_dropdown.value  # Corrected from 'model' to 'model_dropdown.value'
            vae_url = vae_dropdown.value
            print(f"Starting downloads with token: {token_input.value[:4]}...")  # Only show the first 4 characters of the token
            model_result = download_file(model_url)
            vae_result = download_file(vae_url)
            print(model_result)
            print(vae_result)


Dropdown(description='Model:', options={'gsdf/CounterfeitXL': 'https://huggingface.co/gsdf/CounterfeitXL/resol…

Dropdown(description='VAE:', options={'None': '', 'Original VAE': 'https://huggingface.co/stabilityai/sdxl-vae…

Password(description='Token:')

Button(description='Download', style=ButtonStyle())

Output()

In [35]:
from ipywidgets import Button, Text, Output
from IPython.display import display
import os
import shutil

# Setup widgets for input
source_dir_input = Text(value='/mnt/myhdd/train/sungkima', description='Source Directory:')
base_target_dir_input = Text(value='/home/jovyan/train', description='Base Target Directory:')
transfer_button = Button(description='Create Subfolder and Transfer Data')
output_area = Output()

# Display widgets for user interaction
display(source_dir_input, base_target_dir_input, transfer_button, output_area)

def transfer_files(source_dir, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    for filename in os.listdir(source_dir):
        source_file = os.path.join(source_dir, filename)
        target_file = os.path.join(target_dir, filename)
        if not os.path.exists(target_file):
            shutil.copy(source_file, target_file)
        else:
            print(f"File already exists: {target_file}")

@transfer_button.on_click
def on_transfer_clicked(b):
    with output_area:
        output_area.clear_output()
        source_dir = source_dir_input.value
        base_target_dir = base_target_dir_input.value
        subfolder_name = os.path.basename(os.path.normpath(source_dir))
        target_dir = os.path.join(base_target_dir, subfolder_name)
        if not os.path.exists(source_dir):
            print("Source directory does not exist.")
            return
        transfer_files(source_dir, target_dir)
        print(f"Data transferred from {source_dir} to {target_dir}")
        %store subfolder_name


Text(value='/mnt/myhdd/train/sungkima', description='Source Directory:')

Text(value='/home/jovyan/train', description='Base Target Directory:')

Button(description='Create Subfolder and Transfer Data', style=ButtonStyle())

Output()

In [ ]:
%store -r subfolder_name
base_target_dir = '/home/jovyan/train'
train_data_dir = os.path.join(base_target_dir, subfolder_name)

# Now `train_data_dir` can be used in your processing scripts
print("Training data directory:", train_data_dir)


# **III. Data Preprocessing**

In [ ]:
# @title ## **3.1. Data Cleaning**
import os
import random
import concurrent.futures
from tqdm import tqdm
from PIL import Image

%store -r

os.chdir(root_dir)

test = os.listdir(train_data_dir)
#@markdown This section removes unsupported media types such as `.mp4`, `.webm`, and `.gif`, as well as any unnecessary files.
#@markdown To convert a transparent dataset with an alpha channel (RGBA) to RGB and give it a white background, set the `convert` parameter to `True`.
convert = True  # @param {type:"boolean"}
#@markdown Alternatively, you can give the background a `random_color` instead of white by checking the corresponding option.
random_color = True  # @param {type:"boolean"}
recursive = False

batch_size = 32
supported_types = [
    ".png",
    ".jpg",
    ".jpeg",
    ".webp",
    ".bmp",
    ".caption",
    ".npz",
    ".txt",
    ".json",
]

background_colors = [
    (255, 255, 255),
    (0, 0, 0),
    (255, 0, 0),
    (0, 255, 0),
    (0, 0, 255),
    (255, 255, 0),
    (255, 0, 255),
    (0, 255, 255),
]

def clean_directory(directory):
    for item in os.listdir(directory):
        file_path = os.path.join(directory, item)
        if os.path.isfile(file_path):
            file_ext = os.path.splitext(item)[1]
            if file_ext not in supported_types:
                print(f"Deleting file {item} from {directory}")
                os.remove(file_path)
        elif os.path.isdir(file_path) and recursive:
            clean_directory(file_path)

def process_image(image_path):
    img = Image.open(image_path)
    img_dir, image_name = os.path.split(image_path)

    if img.mode in ("RGBA", "LA"):
        if random_color:
            background_color = random.choice(background_colors)
        else:
            background_color = (255, 255, 255)
        bg = Image.new("RGB", img.size, background_color)
        bg.paste(img, mask=img.split()[-1])

        if image_name.endswith(".webp"):
            bg = bg.convert("RGB")
            new_image_path = os.path.join(img_dir, image_name.replace(".webp", ".jpg"))
            bg.save(new_image_path, "JPEG")
            os.remove(image_path)
            print(f" Converted image: {image_name} to {os.path.basename(new_image_path)}")
        else:
            bg.save(image_path, "PNG")
            print(f" Converted image: {image_name}")
    else:
        if image_name.endswith(".webp"):
            new_image_path = os.path.join(img_dir, image_name.replace(".webp", ".jpg"))
            img.save(new_image_path, "JPEG")
            os.remove(image_path)
            print(f" Converted image: {image_name} to {os.path.basename(new_image_path)}")
        else:
            img.save(image_path, "PNG")

def find_images(directory):
    images = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".png") or file.endswith(".webp"):
                images.append(os.path.join(root, file))
    return images

clean_directory(train_data_dir)
images = find_images(train_data_dir)
num_batches = len(images) // batch_size + 1

if convert:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        for i in tqdm(range(num_batches)):
            start = i * batch_size
            end = start + batch_size
            batch = images[start:end]
            executor.map(process_image, batch)

    print("All images have been converted")

100%|██████████| 1/1 [00:00<00:00, 818.56it/s]


All images have been converted


## **3.2. Data Captioning**

- For general images, use BLIP captioning.
- For anime and manga-style images, use Waifu Diffusion 1.4 Tagger V2.

In [ ]:
#@title ### **3.2.1. BLIP Captioning**
#@markdown BLIP is a pre-training framework for unified vision-language understanding and generation, which achieves state-of-the-art results on a wide range of vision-language tasks. It can be used as a tool for image captioning, for example, `astronaut riding a horse in space`.
import os

os.chdir(finetune_dir)

beam_search = True #@param {type:'boolean'}
min_length = 5 #@param {type:"slider", min:0, max:100, step:5.0}
max_length = 90 #@param {type:"slider", min:0, max:100, step:5.0}

config = {
    "_train_data_dir"   : train_data_dir,
    "batch_size"        : 8,
    "beam_search"       : beam_search,
    "min_length"        : min_length,
    "max_length"        : max_length,
    "debug"             : True,
    "caption_extension" : ".caption",
    "max_data_loader_n_workers" : 2,
    "recursive"         : True
}

args = ""
for k, v in config.items():
    if k.startswith("_"):
        args += f'"{v}" '
    elif isinstance(v, str):
        args += f'--{k}="{v}" '
    elif isinstance(v, bool) and v:
        args += f"--{k} "
    elif isinstance(v, float) and not isinstance(v, bool):
        args += f"--{k}={v} "
    elif isinstance(v, int) and not isinstance(v, bool):
        args += f"--{k}={v} "

final_args = f"python make_captions.py {args}"

os.chdir(finetune_dir)
!{final_args}

In [ ]:
import os

# Retrieve the stored training data directory
%store -r train_data_dir

# Colab form fields for interactive input
file_name = "" #@param {type:"string"}
original_text = "a man " #@param {type:"string"}
replacement_text = "whj " #@param {type:"string"}
include_subdirectories = True #@param {type:"boolean"}

def enhanced_text_replacement(directory_path=train_data_dir, file_name="", original_text="Enter original text", replacement_text="Enter replacement text", include_subdirectories=False):
    print("Starting enhanced text replacement utility.")

    processed_files = 0

    if include_subdirectories:
        # Process files in the directory and its subdirectories
        for dirpath, dirnames, files in os.walk(directory_path):
            for f in files:
                if f.endswith('.caption'):
                    full_path = os.path.join(dirpath, f)
                    process_file(full_path, original_text, replacement_text)
                    processed_files += 1
    else:
        if file_name == "":
            # Apply changes to all .caption files in the directory
            files = [f for f in os.listdir(directory_path) if f.endswith('.caption')]
        else:
            # Apply changes to the specified file only
            files = [file_name] if file_name.endswith('.caption') else [file_name + '.caption']

        for filename in files:
            full_path = os.path.join(directory_path, filename)
            process_file(full_path, original_text, replacement_text)
            processed_files += 1

    print(f"Total files updated: {processed_files}")

def process_file(file_path, original_text, replacement_text):
    if os.path.isfile(file_path):
        try:
            with open(file_path, 'r') as file:
                content = file.read()
            content = content.replace(original_text, replacement_text)
            with open(file_path, 'w') as file:
                file.write(content)
            print(f"Processed {file_path}")
            # Print the updated contents of the file
            print(content)
        except Exception as e:
            print(f"An error occurred while processing {file_path}: {e}")
    else:
        print(f"File not found: {file_path}")

# Function call
enhanced_text_replacement(file_name=file_name, original_text=original_text, replacement_text=replacement_text, include_subdirectories=include_subdirectories)


In [ ]:
#@title ### **3.2.2. Waifu Diffusion 1.4 Tagger V2**
import os
%store -r

os.chdir(finetune_dir)

#@markdown [Waifu Diffusion 1.4 Tagger V2](https://huggingface.co/spaces/SmilingWolf/wd-v1-4-tags) is a Danbooru-styled image classification model developed by SmilingWolf. It can also be useful for general image tagging, for example, `1girl, solo, looking_at_viewer, short_hair, bangs, simple_background`.
model = "SmilingWolf/wd-v1-4-convnextv2-tagger-v2" #@param ["SmilingWolf/wd-v1-4-moat-tagger-v2", "SmilingWolf/wd-v1-4-convnextv2-tagger-v2", "SmilingWolf/wd-v1-4-swinv2-tagger-v2", "SmilingWolf/wd-v1-4-convnext-tagger-v2", "SmilingWolf/wd-v1-4-vit-tagger-v2"]
#@markdown Separate `undesired_tags` with comma `(,)` if you want to remove multiple tags, e.g. `1girl,solo,smile`.
undesired_tags = "" #@param {type:'string'}
#@markdown Adjust `general_threshold` for pruning tags (less tags, less flexible). `character_threshold` is useful if you want to train with character tags, e.g. `hakurei reimu`.
general_threshold = 0.75 #@param {type:"slider", min:0, max:1, step:0.05}
character_threshold = 0.7 #@param {type:"slider", min:0, max:1, step:0.05}

config = {
    "_train_data_dir"           : train_data_dir,
    "batch_size"                : 8,
    "repo_id"                   : model,
    "recursive"                 : True,
    "remove_underscore"         : True,
    "general_threshold"         : general_threshold,
    "character_threshold"       : character_threshold,
    "caption_extension"         : ".txt",
    "max_data_loader_n_workers" : 2,
    "debug"                     : True,
    "undesired_tags"            : undesired_tags
}

args = ""
for k, v in config.items():
    if k.startswith("_"):
        args += f'"{v}" '
    elif isinstance(v, str):
        args += f'--{k}="{v}" '
    elif isinstance(v, bool) and v:
        args += f"--{k} "
    elif isinstance(v, float) and not isinstance(v, bool):
        args += f"--{k}={v} "
    elif isinstance(v, int) and not isinstance(v, bool):
        args += f"--{k}={v} "

final_args = f"python tag_images_by_wd14_tagger.py {args}"

os.chdir(finetune_dir)
!{final_args}

# **Adding Text in front**

In [ ]:
#@title ### **Prepend Text to Files Function**
import os

%store -r train_data_dir

include_subdirectories = True #@param {type:"boolean"}
text_to_prepend = "whj, " #@param {type:"string"}

# Function to prepend text to all files in a directory
def prepend_text_to_files(directory_path=train_data_dir, text_to_prepend="Enter text to prepend", include_subdirectories=False):
    print("Starting text prepend utility.")

    processed_files = 0
    if include_subdirectories:
        # Process files in the directory and its subdirectories
        for dirpath, dirnames, files in os.walk(directory_path):
            for filename in files:
                if filename.endswith('.txt'):
                    full_path = os.path.join(dirpath, filename)
                    prepend_text(full_path, text_to_prepend)
                    processed_files += 1
    else:
        # Iterate over each file in the specified directory only
        for filename in os.listdir(directory_path):
            if filename.endswith('.txt'):
                full_path = os.path.join(directory_path, filename)
                prepend_text(full_path, text_to_prepend)
                processed_files += 1

    print(f"All files have been processed. Total files updated: {processed_files}")

def prepend_text(file_path, text_to_prepend):
    try:
        with open(file_path, 'r') as file:
            original_content = file.read()

        new_content = text_to_prepend + original_content
        with open(file_path, 'w') as file:
            file.write(new_content)

        print(f"Processed {file_path}")
        print(f"Prepended text: {text_to_prepend}")
    except IOError as e:
        print(f"An error occurred while processing {file_path}: {e}")

# Run the function with default arguments
prepend_text_to_files(text_to_prepend=text_to_prepend, include_subdirectories=include_subdirectories)


In [ ]:
# @title ### **3.2.3. Custom Caption/Tag**
import os

%store -r

os.chdir(root_dir)

# @markdown Add or remove custom tags here.
extension   = ".txt"  # @param [".txt", ".caption"]
custom_tag  = "circumcised,"  # @param {type:"string"}
# @markdown Use `sub_folder` option to specify a subfolder for multi-concept training.
# @markdown > Specify `--all` to process all subfolders/`recursive`
sub_folder  = "--all" #@param {type: "string"}
# @markdown Enable this to append custom tags at the end of lines.
append      = True  # @param {type:"boolean"}
# @markdown Enable this if you want to remove captions/tags instead.
remove_tag  = False  # @param {type:"boolean"}
recursive   = False

if sub_folder == "":
    image_dir = train_data_dir
elif sub_folder == "--all":
    image_dir = train_data_dir
    recursive = True
elif sub_folder.startswith("/content"):
    image_dir = sub_folder
else:
    image_dir = os.path.join(train_data_dir, sub_folder)
    os.makedirs(image_dir, exist_ok=True)

def read_file(filename):
    with open(filename, "r") as f:
        contents = f.read()
    return contents

def write_file(filename, contents):
    with open(filename, "w") as f:
        f.write(contents)

def process_tags(filename, custom_tag, append, remove_tag):
    contents = read_file(filename)
    tags = [tag.strip() for tag in contents.split(',')]
    custom_tags = [tag.strip() for tag in custom_tag.split(',')]

    for custom_tag in custom_tags:
        custom_tag = custom_tag.replace("_", " ")
        if remove_tag:
            while custom_tag in tags:
                tags.remove(custom_tag)
        else:
            if custom_tag not in tags:
                if append:
                    tags.append(custom_tag)
                else:
                    tags.insert(0, custom_tag)

    contents = ', '.join(tags)
    write_file(filename, contents)

def process_directory(image_dir, tag, append, remove_tag, recursive):
    for filename in os.listdir(image_dir):
        file_path = os.path.join(image_dir, filename)

        if os.path.isdir(file_path) and recursive:
            process_directory(file_path, tag, append, remove_tag, recursive)
        elif filename.endswith(extension):
            process_tags(file_path, tag, append, remove_tag)

tag = custom_tag

if not any(
    [filename.endswith(extension) for filename in os.listdir(image_dir)]
):
    for filename in os.listdir(image_dir):
        if filename.endswith((".png", ".jpg", ".jpeg", ".webp", ".bmp")):
            open(
                os.path.join(image_dir, filename.split(".")[0] + extension),
                "w",
            ).close()

if custom_tag:
    process_directory(image_dir, tag, append, remove_tag, recursive)

# **IV. Training**



In [26]:
%store -r subfolder_name  # Retrieve the stored subfolder name
base_target_dir = '/home/jovyan/train'
train_data_dir = os.path.join(base_target_dir, subfolder_name)
print("Training data directory:", train_data_dir)

# Define JSON paths and configurations
metadata_json = os.path.join(train_data_dir, "meta_clean.json")
bucketing_json = os.path.join(train_data_dir, "meta_lat.json")
bucket_resolution = 1024  # Slider control replaced by direct assignment
mixed_precision = "no"
skip_existing = True
flip_aug = True
clean_caption = True
recursive = True

# Configuration dictionaries for metadata merging and bucket preparation
metadata_config = {
    "_train_data_dir": train_data_dir,
    "_out_json": metadata_json,
    "recursive": recursive,
    "full_path": recursive,
    "clean_caption": clean_caption
}

bucketing_config = {
    "_train_data_dir": train_data_dir,
    "_in_json": metadata_json,
    "_out_json": bucketing_json,
    "_model_name_or_path": 'vae_path' if 'vae_path' in locals() else 'model_path',
    "recursive": recursive,
    "full_path": recursive,
    "flip_aug": flip_aug,
    "skip_existing": skip_existing,
    "batch_size": 4,
    "max_data_loader_n_workers": 2,
    "max_resolution": f"{bucket_resolution}, {bucket_resolution}",
    "mixed_precision": mixed_precision,
}

def generate_args(config):
    args = ""
    for k, v in config.items():
        if k.startswith("_"):
            args += f'"{v}" '
        elif isinstance(v, str):
            args += f'--{k}="{v}" '
        elif isinstance(v, bool) and v:
            args += f"--{k} "
        elif isinstance(v, float) and not isinstance(v, bool):
            args += f"--{k}={v} "
        elif isinstance(v, int) and not isinstance(v, bool):
            args += f"--{k}={v} "
    return args.strip()

merge_metadata_args = generate_args(metadata_config)
prepare_buckets_args = generate_args(bucketing_config)

# Commands to execute Python scripts
merge_metadata_command = f"python merge_all_to_metadata.py {merge_metadata_args}"
prepare_buckets_command = f"python prepare_buckets_latents.py {prepare_buckets_args}"

# Assuming finetune_dir is defined and correctly set
finetune_dir = '/path/to/your/finetuning/scripts'
os.chdir(finetune_dir)  # Change to the directory containing the scripts
!{merge_metadata_command}
time.sleep(1)  # Small delay to ensure filesystem sync
!{prepare_buckets_command}


no stored variable or alias subfolder_name
no stored variable or alias #
no stored variable or alias Retrieve
no stored variable or alias the
no stored variable or alias stored
no stored variable or alias subfolder
no stored variable or alias name


NameError: name 'subfolder_name' is not defined

In [27]:
import ipywidgets as widgets
from IPython.display import display
import ast
import toml

# Dropdown for selecting network category
network_category_dropdown = widgets.Dropdown(
    options=["LoRA_LierLa", "LoRA_C3Lier", "DyLoRA_LierLa", "DyLoRA_C3Lier", "LoCon", "LoHa", "IA3", "LoKR", "DyLoRA_Lycoris"],
    value='LoRA_LierLa',
    description='Network Category:',
    style={'description_width': 'initial'}
)

# Text input for optional training arguments
network_args_input = widgets.Text(
    value='',
    placeholder='Enter args like ["args=1", "args=2"]',
    description='Network Args:',
    style={'description_width': 'initial'}
)

# Numeric input fields for network and convolution parameters
network_dim_input = widgets.IntText(value=64, description='Network Dim:', style={'description_width': 'initial'})
network_alpha_input = widgets.IntText(value=32, description='Network Alpha:', style={'description_width': 'initial'})
conv_dim_input = widgets.IntText(value=8, description='Conv Dim:', style={'description_width': 'initial'})
conv_alpha_input = widgets.IntText(value=1, description='Conv Alpha:', style={'description_width': 'initial'})
unit_input = widgets.IntText(value=2, description='Unit:', style={'description_width': 'initial'})

# Button to process the inputs and generate configuration
process_button = widgets.Button(description="Generate Configuration")
output_area = widgets.Output()

display(network_category_dropdown, network_args_input, network_dim_input, network_alpha_input, conv_dim_input, conv_alpha_input, unit_input, process_button, output_area)

def process_configuration(b):
    network_args = network_args_input.value.strip()
    if network_args.startswith('[') and network_args.endswith(']'):
        try:
            network_args = ast.literal_eval(network_args)
        except (SyntaxError, ValueError) as e:
            network_args = []
            print(f"Error parsing network_args: {e}\n")
    elif len(network_args) > 0:
        print(f"WARNING! '{network_args}' is not a valid list! Put args like this: [\"args=1\", \"args=2\"]\n")
        network_args = []
    else:
        network_args = []

    # Define network configuration based on inputs
    network_category = network_category_dropdown.value
    network_dim = network_dim_input.value
    network_alpha = network_alpha_input.value
    conv_dim = conv_dim_input.value
    conv_alpha = conv_alpha_input.value
    unit = unit_input.value

    # Configuration logic
    network_config = {
        "LoRA_LierLa": {"module": "networks.lora", "args": []},
        "LoRA_C3Lier": {"module": "networks.lora", "args": [f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}"]},
        # Additional configurations as per original script
    }
    network_module = network_config[network_category]["module"]
    network_args.extend(network_config[network_category]["args"])

    lora_config = {
        "additional_network_arguments": {
            "no_metadata": False,
            "network_module": network_module,
            "network_dim": network_dim,
            "network_alpha": network_alpha,
            "network_args": network_args,
            "network_train_unet_only": True,
            "training_comment": None,
        }
    }

    with output_area:
        output_area.clear_output()
        print(toml.dumps(lora_config))

process_button.on_click(process_configuration)


Dropdown(description='Network Category:', options=('LoRA_LierLa', 'LoRA_C3Lier', 'DyLoRA_LierLa', 'DyLoRA_C3Li…

Text(value='', description='Network Args:', placeholder='Enter args like ["args=1", "args=2"]', style=TextStyl…

IntText(value=64, description='Network Dim:', style=DescriptionStyle(description_width='initial'))

IntText(value=32, description='Network Alpha:', style=DescriptionStyle(description_width='initial'))

IntText(value=8, description='Conv Dim:', style=DescriptionStyle(description_width='initial'))

IntText(value=1, description='Conv Alpha:', style=DescriptionStyle(description_width='initial'))

IntText(value=2, description='Unit:', style=DescriptionStyle(description_width='initial'))

Button(description='Generate Configuration', style=ButtonStyle())

Output()

In [28]:
import ipywidgets as widgets
from IPython.display import display
import ast
import toml

# Dropdown for selecting optimizer type
optimizer_type_dropdown = widgets.Dropdown(
    options=["AdamW", "AdamW8bit", "Lion8bit", "Lion", "SGDNesterov", "SGDNesterov8bit", "DAdaptation(DAdaptAdamPreprint)", "DAdaptAdaGrad", "DAdaptAdam", "DAdaptAdan", "DAdaptAdanIP", "DAdaptLion", "DAdaptSGD", "AdaFactor"],
    value='AdaFactor',
    description='Optimizer Type:',
    style={'description_width': 'initial'}
)

# Text input for additional optimizer arguments
optimizer_args_input = widgets.Text(
    value='[ "scale_parameter=False", "relative_step=False", "warmup_init=False" ]',
    placeholder='Enter args like ["arg1=value", "arg2=value"]',
    description='Optimizer Args:',
    style={'description_width': 'initial'}
)

# Numeric inputs for learning rate and gradient norm configuration
learning_rate_input = widgets.FloatText(value=8e-6, description='Learning Rate:', style={'description_width': 'initial'})
max_grad_norm_input = widgets.FloatText(value=1, description='Max Grad Norm:', style={'description_width': 'initial'})

# Dropdown for selecting the learning rate scheduler
lr_scheduler_dropdown = widgets.Dropdown(
    options=["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup", "adafactor"],
    value='polynomial',
    description='LR Scheduler:',
    style={'description_width': 'initial'}
)

# Numeric input for LR warmup steps and scheduler number
lr_warmup_steps_input = widgets.IntText(value=300, description='LR Warmup Steps:', style={'description_width': 'initial'})
lr_scheduler_num_input = widgets.IntText(value=3, description='LR Scheduler Num:', style={'description_width': 'initial'})

# Button to process the inputs and generate configuration
process_button = widgets.Button(description="Generate Configuration")
output_area = widgets.Output()

display(optimizer_type_dropdown, optimizer_args_input, learning_rate_input, max_grad_norm_input, lr_scheduler_dropdown, lr_warmup_steps_input, lr_scheduler_num_input, process_button, output_area)

def process_configuration(b):
    # Parse optimizer arguments from string to list
    optimizer_args = optimizer_args_input.value.strip()
    if optimizer_args.startswith('[') and optimizer_args.endswith(']'):
        try:
            optimizer_args = ast.literal_eval(optimizer_args)
        except (SyntaxError, ValueError) as e:
            print(f"Error parsing optimizer_args: {e}")
            optimizer_args = []
    else:
        optimizer_args = []

    # Gather all configurations
    optimizer_config = {
        "optimizer_arguments": {
            "optimizer_type": optimizer_type_dropdown.value,
            "learning_rate": learning_rate_input.value,
            "max_grad_norm": max_grad_norm_input.value,
            "optimizer_args": optimizer_args,
            "lr_scheduler": lr_scheduler_dropdown.value,
            "lr_warmup_steps": lr_warmup_steps_input.value,
            "lr_scheduler_num_cycles": lr_scheduler_num_input.value if lr_scheduler_dropdown.value == "cosine_with_restarts" else None,
            "lr_scheduler_power": lr_scheduler_num_input.value if lr_scheduler_dropdown.value == "polynomial" else None,
        },
    }

    with output_area:
        output_area.clear_output()
        print(toml.dumps(optimizer_config))

process_button.on_click(process_configuration)


Dropdown(description='Optimizer Type:', index=13, options=('AdamW', 'AdamW8bit', 'Lion8bit', 'Lion', 'SGDNeste…

Text(value='[ "scale_parameter=False", "relative_step=False", "warmup_init=False" ]', description='Optimizer A…

FloatText(value=8e-06, description='Learning Rate:', style=DescriptionStyle(description_width='initial'))

FloatText(value=1.0, description='Max Grad Norm:', style=DescriptionStyle(description_width='initial'))

Dropdown(description='LR Scheduler:', index=3, options=('linear', 'cosine', 'cosine_with_restarts', 'polynomia…

IntText(value=300, description='LR Warmup Steps:', style=DescriptionStyle(description_width='initial'))

IntText(value=3, description='LR Scheduler Num:', style=DescriptionStyle(description_width='initial'))

Button(description='Generate Configuration', style=ButtonStyle())

Output()

In [29]:
import ipywidgets as widgets
from IPython.display import display
import toml

# Checkbox for saving/loading optimizer state
save_optimizer_state_checkbox = widgets.Checkbox(value=True, description='Save Optimizer State', disabled=False)
load_optimizer_state_input = widgets.Text(value='/content/drive/MyDrive/kohya-trainer/output/whj2/whj2-state', description='Load State Path:', style={'description_width': 'initial'})

# Dropdown for noise control type
noise_control_dropdown = widgets.Dropdown(
    options=['none', 'noise_offset', 'multires_noise'],
    value='multires_noise',
    description='Noise Control Type:',
    style={'description_width': 'initial'}
)

# Numeric input for noise offset and adaptive scale
noise_offset_input = widgets.FloatText(value=0.1, description='Noise Offset:', style={'description_width': 'initial'})
adaptive_noise_scale_input = widgets.FloatText(value=0.01, description='Adaptive Noise Scale:', style={'description_width': 'initial'})

# Slider for multires noise iterations and discount
multires_noise_iterations_slider = widgets.IntSlider(value=10, min=1, max=10, step=1, description='Multires Iterations:')
multires_noise_discount_slider = widgets.FloatSlider(value=0.8, min=0.1, max=1, step=0.1, description='Multires Discount:')

# Caption dropout configuration
caption_dropout_rate_input = widgets.FloatText(value=0.1, description='Caption Dropout Rate:', style={'description_width': 'initial'})
caption_tag_dropout_rate_input = widgets.FloatText(value=0.1, description='Tag Dropout Rate:', style={'description_width': 'initial'})
caption_dropout_every_n_epochs_input = widgets.IntText(value=5, description='Dropout Every N Epochs:', style={'description_width': 'initial'})

# Gamma configuration
min_snr_gamma_input = widgets.FloatText(value=5, description='Min SNR Gamma:', style={'description_width': 'initial'})

# Button to process the inputs and generate configuration
process_button = widgets.Button(description="Generate Configuration")
output_area = widgets.Output()

# Display all widgets
display(save_optimizer_state_checkbox, load_optimizer_state_input, noise_control_dropdown, 
        noise_offset_input, adaptive_noise_scale_input, multires_noise_iterations_slider, 
        multires_noise_discount_slider, caption_dropout_rate_input, caption_tag_dropout_rate_input, 
        caption_dropout_every_n_epochs_input, min_snr_gamma_input, process_button, output_area)

def process_configuration(b):
    advanced_training_config = {
        "advanced_training_config": {
            "resume": load_optimizer_state_input.value,
            "save_state": save_optimizer_state_checkbox.value,
            "save_last_n_epochs_state": save_optimizer_state_checkbox.value,
            "noise_offset": noise_offset_input.value if noise_control_dropdown.value == "noise_offset" else None,
            "adaptive_noise_scale": adaptive_noise_scale_input.value if adaptive_noise_scale_input.value and noise_control_dropdown.value == "noise_offset" else None,
            "multires_noise_iterations": multires_noise_iterations_slider.value if noise_control_dropdown.value == "multires_noise" else None,
            "multires_noise_discount": multires_noise_discount_slider.value if noise_control_dropdown.value == "multires_noise" else None,
            "caption_dropout_rate": caption_dropout_rate_input.value,
            "caption_tag_dropout_rate": caption_tag_dropout_rate_input.value,
            "caption_dropout_every_n_epochs": caption_dropout_every_n_epochs_input.value,
            "min_snr_gamma": min_snr_gamma_input.value if min_snr_gamma_input.value != -1 else None,
        }
    }

    with output_area:
        output_area.clear_output()
        print(toml.dumps(advanced_training_config))

process_button.on_click(process_configuration)


Checkbox(value=True, description='Save Optimizer State')

Text(value='/content/drive/MyDrive/kohya-trainer/output/whj2/whj2-state', description='Load State Path:', styl…

Dropdown(description='Noise Control Type:', index=2, options=('none', 'noise_offset', 'multires_noise'), style…

FloatText(value=0.1, description='Noise Offset:', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.01, description='Adaptive Noise Scale:', style=DescriptionStyle(description_width='initial')…

...

FloatText(value=0.1, description='Tag Dropout Rate:', style=DescriptionStyle(description_width='initial'))

IntText(value=5, description='Dropout Every N Epochs:', style=DescriptionStyle(description_width='initial'))

FloatText(value=5.0, description='Min SNR Gamma:', style=DescriptionStyle(description_width='initial'))

Button(description='Generate Configuration', style=ButtonStyle())

Output()

In [33]:
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import ipywidgets as widgets
from IPython.display import display

# Attempt to retrieve subfolder_name, ensure you have run the %store subfolder_name in another cell before this.
try:
    %store -r subfolder_name
except NameError:
    print("subfolder_name is not defined. Ensure you have stored it with '%store subfolder_name' after setting it.")
    subfolder_name = "default_subfolder"  # Provide a default or an error handling routine

base_target_dir = '/home/jovyan/train'
train_data_dir = os.path.join(base_target_dir, subfolder_name)

class SingleFolderDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.images = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith((".png", ".jpg", ".jpeg"))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Define transformations and DataLoader settings
transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),
    transforms.ToTensor(),
])

# Widgets for data loader configuration
batch_size_input = widgets.IntText(value=20, description='Batch Size:', style={'description_width': 'initial'})
shuffle_checkbox = widgets.Checkbox(value=True, description='Shuffle:', disabled=False)
num_workers_input = widgets.IntText(value=4, description='Num Workers:', style={'description_width': 'initial'})
init_button = widgets.Button(description="Initialize Dataset and Loader")
output_area = widgets.Output()

display(batch_size_input, shuffle_checkbox, num_workers_input, init_button, output_area)

def initialize_dataset_and_loader(b):
    dataset = SingleFolderDataset(directory=train_data_dir, transform=transformations)
    data_loader = DataLoader(dataset, batch_size=batch_size_input.value, shuffle=shuffle_checkbox.value, num_workers=num_workers_input.value)
    with output_area:
        output_area.clear_output()
        print(f"Dataset initialized with {len(dataset)} images.")
        print(f"Data loader ready with batch size {batch_size_input.value}.")

init_button.on_click(initialize_dataset_and_loader)


no stored variable or alias subfolder_name


NameError: name 'subfolder_name' is not defined

In [36]:
import ipywidgets as widgets
from IPython.display import display
import toml
import os
from subprocess import getoutput
import textwrap
import random
import glob

# General Project Configurations
project_name_input = widgets.Text(value='whj3', description='Project Name:', style={'description_width': 'initial'})
wandb_api_key_input = widgets.Text(value='855a9e4f2b1aa8b2e2e27e9c1c1373071fb23c42', description='WANDB API Key:', style={'description_width': 'initial'})
in_json_input = widgets.Text(value='/content/LoRA/meta_lat.json', description='Input JSON Path:', style={'description_width': 'initial'})

# SDXL Configurations
gradient_checkpointing_checkbox = widgets.Checkbox(value=True, description='Gradient Checkpointing', disabled=False)
no_half_vae_checkbox = widgets.Checkbox(value=True, description='No Half VAE', disabled=False)
cache_text_encoder_outputs_checkbox = widgets.Checkbox(value=True, description='Cache Text Encoder Outputs', disabled=False)

# Dataset Configurations
num_repeats_input = widgets.IntText(value=20, description='Num Repeats:', style={'description_width': 'initial'})
resolution_slider = widgets.IntSlider(value=1024, min=512, max=1024, step=128, description='Resolution:')
keep_tokens_input = widgets.IntText(value=0, description='Keep Tokens:', style={'description_width': 'initial'})

# Noise Control Configurations
noise_control_type_dropdown = widgets.Dropdown(
    options=['none', 'noise_offset', 'multires_noise'],
    value='multires_noise',
    description='Noise Control Type:',
    style={'description_width': 'initial'}
)
noise_offset_num_input = widgets.FloatText(value=0.1, description='Noise Offset Num:', style={'description_width': 'initial'})
adaptive_noise_scale_input = widgets.FloatText(value=0.01, description='Adaptive Noise Scale:', style={'description_width': 'initial'})

# Training Specific Configurations
min_timestep_input = widgets.IntText(value=100, description='Min Timestep:', style={'description_width': 'initial'})
max_timestep_input = widgets.IntText(value=1000, description='Max Timestep:', style={'description_width': 'initial'})
train_batch_size_input = widgets.IntText(value=20, description='Train Batch Size:', style={'description_width': 'initial'})
mixed_precision_dropdown = widgets.Dropdown(
    options=["no", "fp16", "bf16"],
    value='bf16',
    description='Mixed Precision:',
    style={'description_width': 'initial'}
)
num_epochs_input = widgets.IntText(value=5, description='Num Epochs:', style={'description_width': 'initial'})

# Save Configurations
save_precision_dropdown = widgets.Dropdown(
    options=["float", "fp16", "bf16"],
    value='bf16',
    description='Save Precision:',
    style={'description_width': 'initial'}
)
save_every_n_epochs_input = widgets.IntText(value=1, description='Save Every N Epochs:', style={'description_width': 'initial'})

# Sample Prompt Configurations
enable_sample_checkbox = widgets.Checkbox(value=True, description='Enable Sample', disabled=False)
sampler_dropdown = widgets.Dropdown(
    options=["ddim", "pndm", "lms", "euler", "euler_a", "heun", "dpm_2", "dpm_2_a", "dpmsolver", "dpmsolver++", "dpmsingle", "k_lms", "k_euler", "k_euler_a", "k_dpm_2", "k_dpm_2_a"],
    value='euler_a',
    description='Sampler:',
    style={'description_width': 'initial'}
)

# Button to generate configuration
generate_config_button = widgets.Button(description="Generate Training Configuration")
output_area = widgets.Output()

# Display the widgets
accordion = widgets.Accordion(children=[
    widgets.VBox([project_name_input, wandb_api_key_input, in_json_input]),
    widgets.VBox([gradient_checkpointing_checkbox, no_half_vae_checkbox, cache_text_encoder_outputs_checkbox]),
    widgets.VBox([num_repeats_input, resolution_slider, keep_tokens_input]),
    widgets.VBox([noise_control_type_dropdown, noise_offset_num_input, adaptive_noise_scale_input]),
    widgets.VBox([min_timestep_input, max_timestep_input, train_batch_size_input, mixed_precision_dropdown, num_epochs_input]),
    widgets.VBox([save_precision_dropdown, save_every_n_epochs_input, enable_sample_checkbox, sampler_dropdown])
])
accordion.set_title(0, 'Project Configurations')
accordion.set_title(1, 'SDXL Configurations')
accordion.set_title(2, 'Dataset Configurations')
accordion.set_title(3, 'Noise Control Configurations')
accordion.set_title(4, 'Training Specific Configurations')
accordion.set_title(5, 'Save & Sample Configurations')

display(accordion, generate_config_button, output_area)

def generate_config(b):
    # Prepare configuration dictionary based on the inputs
    training_config = {
        'project_name': project_name_input.value,
        'wandb_api_key': wandb_api_key_input.value,
        'input_json': in_json_input.value,
        'gradient_checkpointing': gradient_checkpointing_checkbox.value,
        'no_half_vae': no_half_vae_checkbox.value,
        'cache_text_encoder_outputs': cache_text_encoder_outputs_checkbox.value,
        'num_repeats': num_repeats_input.value,
        'resolution': resolution_slider.value,
        'keep_tokens': keep_tokens_input.value,
        'noise_control_type': noise_control_type_dropdown.value,
        'noise_offset_num': noise_offset_num_input.value,
        'adaptive_noise_scale': adaptive_noise_scale_input.value,
        'min_timestep': min_timestep_input.value,
        'max_timestep': max_timestep_input.value,
        'train_batch_size': train_batch_size_input.value,
        'mixed_precision': mixed_precision_dropdown.value,
        'num_epochs': num_epochs_input.value,
        'save_precision': save_precision_dropdown.value,
        'save_every_n_epochs': save_every_n_epochs_input.value,
        'enable_sample': enable_sample_checkbox.value,
        'sampler': sampler_dropdown.value
    }

    config_toml = toml.dumps(training_config)
    
    with output_area:
        output_area.clear_output()
        print("Training Configuration:\n")
        print(config_toml)

generate_config_button.on_click(generate_config)


Accordion(children=(VBox(children=(Text(value='whj3', description='Project Name:', style=TextStyle(description…

Button(description='Generate Training Configuration', style=ButtonStyle())

Output()

In [ ]:

#@title ## **4.5. Start Training**
import os
import toml

#@markdown Check your config here if you want to edit something:
#@markdown - `sample_prompt` : /content/LoRA/config/sample_prompt.toml
#@markdown - `config_file` : /content/LoRA/config/config_file.toml


#@markdown You can import config from another session if you want.

sample_prompt   = "/content/LoRA/config/sample_prompt.toml" #@param {type:'string'}
config_file     = "/content/LoRA/config/config_file.toml" #@param {type:'string'}

def read_file(filename):
    with open(filename, "r") as f:
        contents = f.read()
    return contents

def train(config):
    args = ""
    for k, v in config.items():
        if k.startswith("_"):
            args += f'"{v}" '
        elif isinstance(v, str):
            args += f'--{k}="{v}" '
        elif isinstance(v, bool) and v:
            args += f"--{k} "
        elif isinstance(v, float) and not isinstance(v, bool):
            args += f"--{k}={v} "
        elif isinstance(v, int) and not isinstance(v, bool):
            args += f"--{k}={v} "

    return args

accelerate_conf = {
    "config_file" : "/content/kohya-trainer/accelerate_config/config.yaml",
    "num_cpu_threads_per_process" : 1,
}

train_conf = {
    "sample_prompts"  : sample_prompt if os.path.exists(sample_prompt) else None,
    "config_file"     : config_file,
    "wandb_api_key"   : wandb_api_key if wandb_api_key else None
}

accelerate_args = train(accelerate_conf)
train_args = train(train_conf)

final_args = f"accelerate launch {accelerate_args} sdxl_train_network.py {train_args}"

os.chdir(repo_dir)
!{final_args}

Loading settings from /content/LoRA/config/config_file.toml...
/content/LoRA/config/config_file
prepare tokenizers
update token length: 225
Training with captions.
loading existing metadata: /content/LoRA/meta_lat.json
metadata has bucket info, enable bucketing / メタデータにbucket情報があるためbucketを有効にします
using bucket info in metadata / メタデータ内のbucket情報を使います
[Dataset 0]
  batch_size: 20
  resolution: (1024, 1024)
  enable_bucket: True
  min_bucket_reso: None
  max_bucket_reso: None
  bucket_reso_steps: None
  bucket_no_upscale: None

  [Subset 0 of Dataset 0]
    image_dir: "/content/drive/MyDrive/train/WHJ4"
    image_count: 66
    num_repeats: 20
    shuffle_caption: False
    keep_tokens: 0
    caption_dropout_rate: 0.0
    caption_dropout_every_n_epoches: 0
    caption_tag_dropout_rate: 0.0
    color_aug: False
    flip_aug: False
    face_crop_aug_range: None
    random_crop: False
    token_warmup_min: 1,
    token_warmup_step: 0,
    metadata_file: /content/LoRA/meta_lat.json


[Dataset 0]